In [ ]:
import ipyvolume as ipv
import itkwidgets as itkw
import numpy as np
from nanomesh import NanoMesher
from nanomesh import sitk_filter
from nanomesh.utils import show_slice, show_volume, generate_mesh_from_binary_image

In [ ]:
mesh = NanoMesher()
mesh.load_bin("sample_data.vol",size=[200,200,200], input_dtype=np.uint8, output_dtype=np.float32)
show_slice(mesh.volume.img,'z',scale=1)

In [ ]:
mesh.apply_gaussian_filtering(sigma=5.)
show_slice(mesh.volume.img,'z')

In [ ]:
show_volume(mesh.volume.img)

In [ ]:
inner = sitk_filter.binary_threshold(mesh.volume.img, lowerThreshold=0, upperThreshold=125, insideValue=1, outsideValue=0)
show_slice((mesh.volume.img, inner), dim='z', title="Inner regions")

In [ ]:
show_volume(inner)

In [ ]:
inner_mesh = generate_mesh_from_binary_image(inner)

In [ ]:
import pyvista as pv
pv.plot_itk(inner_mesh)

In [ ]:
import meshio
meshio.write('inner.vtk',inner_mesh)

In [ ]:
inner_mesh = pv.read('inner.vtk')

bounds = [100,200,100,200,100,200]
clipped = inner_mesh.clip_box(bounds)

pv.set_plot_theme("document")

p = pv.Plotter(notebook=True)

p.add_mesh(mesh.volume.img, style='wireframe', color='black')
p.add_mesh(clipped,  color='lightgrey')
p.show()

In [ ]:
two_phase = sitk_filter.binary_threshold(mesh.volume.img, lowerThreshold=0, upperThreshold=125, insideValue=1, outsideValue=2)
show_slice((mesh.volume.img, two_phase), dim='z', title="Binary Thresholding")

In [ ]:
show_volume(two_phase)

In [ ]:
two_phase_mesh = generate_mesh_from_binary_image(two_phase)

In [ ]:
pv.plot_itk(two_phase_mesh)

In [ ]:
import meshio
meshio.write('twophase.vtk', two_phase_mesh)

In [ ]:

mesh = pv.read('twophase.vtk')

bounds = [100,200,100,200,100,200]
clipped = mesh.clip_box(bounds)
pv.set_plot_theme("document")

p = pv.Plotter(notebook=True)
p.add_mesh(mesh, style='wireframe', color='black')
p.add_mesh(clipped)
p.show(use_ipyvtk=False)
